In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv("/kaggle/input/playground-series-s3e3/train.csv")
train.head()

,id,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EnvironmentSatisfaction,...,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Attrition
0,0,36,Travel_Frequently,599,Research & Development,24,3,Medical,1,4,...,80,1,10,2,3,10,0,7,8,0
1,1,35,Travel_Rarely,921,Sales,8,3,Other,1,1,...,80,1,4,3,3,4,2,0,3,0
2,2,32,Travel_Rarely,718,Sales,26,3,Marketing,1,3,...,80,2,4,3,3,3,2,1,2,0
3,3,38,Travel_Rarely,1488,Research & Development,2,3,Medical,1,3,...,80,0,15,1,1,6,0,0,2,0
4,4,50,Travel_Rarely,1017,Research & Development,5,4,Medical,1,2,...,80,0,31,0,3,31,14,4,10,1


In [3]:
test = pd.read_csv("/kaggle/input/playground-series-s3e3/test.csv")
test.head()

,id,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EnvironmentSatisfaction,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,1677,19,Non-Travel,992,Research & Development,1,1,Medical,1,4,...,4,80,0,1,2,2,1,0,0,0
1,1678,45,Travel_Rarely,1136,Sales,4,4,Marketing,1,3,...,3,80,1,7,3,3,2,2,2,2
2,1679,37,Travel_Rarely,155,Research & Development,13,3,Life Sciences,1,4,...,2,80,2,13,2,2,7,7,1,7
3,1680,32,Travel_Rarely,688,Research & Development,1,4,Life Sciences,1,3,...,3,80,0,14,2,2,14,10,11,8
4,1681,29,Travel_Frequently,464,Research & Development,9,1,Life Sciences,1,3,...,1,80,0,1,5,3,1,0,0,0


In [4]:
train.shape, test.shape

((1677, 35), (1119, 34))

In [5]:
train.isna().sum()

id                          0
Age                         0
BusinessTravel              0
DailyRate                   0
Department                  0
DistanceFromHome            0
Education                   0
EducationField              0
EmployeeCount               0
EnvironmentSatisfaction     0
Gender                      0
HourlyRate                  0
JobInvolvement              0
JobLevel                    0
JobRole                     0
JobSatisfaction             0
MaritalStatus               0
MonthlyIncome               0
MonthlyRate                 0
NumCompaniesWorked          0
Over18                      0
OverTime                    0
PercentSalaryHike           0
PerformanceRating           0
RelationshipSatisfaction    0
StandardHours               0
StockOptionLevel            0
TotalWorkingYears           0
TrainingTimesLastYear       0
WorkLifeBalance             0
YearsAtCompany              0
YearsInCurrentRole          0
YearsSinceLastPromotion     0
YearsWithC

In [6]:
test.isna().sum()

id                          0
Age                         0
BusinessTravel              0
DailyRate                   0
Department                  0
DistanceFromHome            0
Education                   0
EducationField              0
EmployeeCount               0
EnvironmentSatisfaction     0
Gender                      0
HourlyRate                  0
JobInvolvement              0
JobLevel                    0
JobRole                     0
JobSatisfaction             0
MaritalStatus               0
MonthlyIncome               0
MonthlyRate                 0
NumCompaniesWorked          0
Over18                      0
OverTime                    0
PercentSalaryHike           0
PerformanceRating           0
RelationshipSatisfaction    0
StandardHours               0
StockOptionLevel            0
TotalWorkingYears           0
TrainingTimesLastYear       0
WorkLifeBalance             0
YearsAtCompany              0
YearsInCurrentRole          0
YearsSinceLastPromotion     0
YearsWithC

In [7]:
# eliminating annoying lgbm warnings, source: https://stackoverflow.com/a/33616192/1105837
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

from lightgbm.sklearn import LGBMClassifier
import lightgbm as lgbm
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold

In [8]:
original_data = pd.read_csv('/kaggle/input/ibm-hr-analytics-attrition-dataset/WA_Fn-UseC_-HR-Employee-Attrition.csv')

# munging the original data to make it appear more like our train DataFrame
original_data = original_data.rename(columns={'EmployeeNumber': 'id'})
original_data['Attrition'] = (original_data['Attrition'] == 'Yes').astype(int)

# rearranging columns so that they are in the same order as in train
original_data = original_data[train.columns.tolist()]

# adding the source column
original_data['source'] = 'original'
train['source'] = 'synthetic'
test['source'] = 'synthetic'

# combining the datasets
train = pd.concat([train, original_data])
train.reset_index(inplace=True, drop=True)
train.shape

(3147, 36)

In [9]:
train.head()

,id,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EnvironmentSatisfaction,...,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Attrition,source
0,0,36,Travel_Frequently,599,Research & Development,24,3,Medical,1,4,...,1,10,2,3,10,0,7,8,0,synthetic
1,1,35,Travel_Rarely,921,Sales,8,3,Other,1,1,...,1,4,3,3,4,2,0,3,0,synthetic
2,2,32,Travel_Rarely,718,Sales,26,3,Marketing,1,3,...,2,4,3,3,3,2,1,2,0,synthetic
3,3,38,Travel_Rarely,1488,Research & Development,2,3,Medical,1,3,...,0,15,1,1,6,0,0,2,0,synthetic
4,4,50,Travel_Rarely,1017,Research & Development,5,4,Medical,1,2,...,0,31,0,3,31,14,4,10,1,synthetic


In [10]:
train.tail()

,id,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EnvironmentSatisfaction,...,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Attrition,source
3142,2061,36,Travel_Frequently,884,Research & Development,23,2,Medical,1,3,...,1,17,3,3,5,2,0,3,0,original
3143,2062,39,Travel_Rarely,613,Research & Development,6,1,Medical,1,4,...,1,9,5,3,7,7,1,7,0,original
3144,2064,27,Travel_Rarely,155,Research & Development,4,3,Life Sciences,1,2,...,1,6,0,3,6,2,0,3,0,original
3145,2065,49,Travel_Frequently,1023,Sales,2,3,Medical,1,4,...,0,17,3,2,9,6,0,8,0,original
3146,2068,34,Travel_Rarely,628,Research & Development,8,3,Medical,1,2,...,0,6,3,4,4,3,1,2,0,original


In [11]:
features = list(train.columns)
features.remove('id')
features.remove('Attrition')

target = 'Attrition'

In [12]:
def woe_iv(df, var, target):
    p1, p2, p3, p4, p5, p6, p7, p8, p9 = df[var].quantile([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])
    bins = [p1, p2, p3, p4, p5, p6, p7, p8, p9]
    labels = [1,2,3,4,5,6,7,8]
    temp = pd.DataFrame(columns=['binned', 'target', 'var_woe'])
    temp['binned'] = pd.cut(df[var], bins=bins, labels=labels)
    temp['target'] = df[target]
    temp_1 = temp.groupby(['binned', 'target'])['target'].count().reset_index(name='counts')
    temp_yes = temp_1.loc[(temp_1['target'] == 1)]
    temp_no = temp_1.loc[(temp_1['target'] == 0)]
    temp_2 = temp_yes.merge(temp_no,on='binned')
    temp_2.columns = ['bin', 'yes', 'count_yes', 'no', 'count_no']
    sum_yes = np.sum(temp_2['count_yes'])
    sum_no = np.sum(temp_2['count_no'])
    temp_2['GoodPer']  = 0.0
    temp_2['BadPer']  = 0.0
    temp_2['woe'] = 0.0
    temp_2['iv'] = 0.0
    for i in range(len(temp_2)):
        temp_2['GoodPer'][i] = (temp_2['count_yes'][i] / sum_yes)
        #print (temp_2['GoodPer'][i], temp_2['count_yes'][i] , sum_yes, (temp_2['count_yes'][i] / sum_yes))
        temp_2['BadPer'][i] = (temp_2['count_no'][i] / sum_no)
        temp_2['woe'][i] = np.log(temp_2['GoodPer'][i] / temp_2['BadPer'][i])
        temp_2['iv'][i] = (temp_2['GoodPer'][i] - temp_2['BadPer'][i]) * temp_2['woe'][i]
    print (np.sum(temp_2['iv']))
    for j in range(len(temp)):
        if (temp.loc[j, 'binned'] == 1):
            temp.loc[j, 'var_woe'] = temp_2.loc[0, 'woe']
        elif (temp.loc[j, 'binned'] == 2):
            temp.loc[j, 'var_woe'] = temp_2.loc[1, 'woe']
        elif (temp.loc[j, 'binned'] == 3):
            temp.loc[j, 'var_woe'] = temp_2.loc[2, 'woe']
        elif (temp.loc[j, 'binned'] == 4):
            temp.loc[j, 'var_woe'] = temp_2.loc[3, 'woe']
        elif (temp.loc[j, 'binned'] == 5):
            temp.loc[j, 'var_woe'] = temp_2.loc[4, 'woe']
        elif (temp.loc[j, 'binned'] == 6):
            temp.loc[j, 'var_woe'] = temp_2.loc[5, 'woe']
        elif (temp.loc[j, 'binned'] == 7):
            temp.loc[j, 'var_woe'] = temp_2.loc[6, 'woe']
        elif (temp.loc[j, 'binned'] == 8):
            temp.loc[j, 'var_woe'] = temp_2.loc[7, 'woe']
        else:
            temp.loc[j, 'var_woe'] = 0
    return temp

In [13]:
train.columns

Index(['id', 'Age', 'BusinessTravel', 'DailyRate', 'Department',
       'DistanceFromHome', 'Education', 'EducationField', 'EmployeeCount',
       'EnvironmentSatisfaction', 'Gender', 'HourlyRate', 'JobInvolvement',
       'JobLevel', 'JobRole', 'JobSatisfaction', 'MaritalStatus',
       'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked', 'Over18',
       'OverTime', 'PercentSalaryHike', 'PerformanceRating',
       'RelationshipSatisfaction', 'StandardHours', 'StockOptionLevel',
       'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance',
       'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion',
       'YearsWithCurrManager', 'Attrition', 'source'],
      dtype='object')

In [14]:
tmp = woe_iv(train, 'Age', 'Attrition')
tmp.head()

0.21242627391169555


,binned,target,var_woe
0,5,0,-0.338715
1,4,0,-0.101807
2,3,0,-0.020844
3,5,0,-0.338715
4,NaN,1,0


In [15]:
train['Age'] = tmp['var_woe']
train.head()

,id,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EnvironmentSatisfaction,...,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Attrition,source
0,0,-0.338715,Travel_Frequently,599,Research & Development,24,3,Medical,1,4,...,1,10,2,3,10,0,7,8,0,synthetic
1,1,-0.101807,Travel_Rarely,921,Sales,8,3,Other,1,1,...,1,4,3,3,4,2,0,3,0,synthetic
2,2,-0.020844,Travel_Rarely,718,Sales,26,3,Marketing,1,3,...,2,4,3,3,3,2,1,2,0,synthetic
3,3,-0.338715,Travel_Rarely,1488,Research & Development,2,3,Medical,1,3,...,0,15,1,1,6,0,0,2,0,synthetic
4,4,0,Travel_Rarely,1017,Research & Development,5,4,Medical,1,2,...,0,31,0,3,31,14,4,10,1,synthetic


In [16]:
for ii in train.columns:
    if type(train[ii][0]) == str:
        print (ii)

BusinessTravel
Department
EducationField
Gender
JobRole
MaritalStatus
Over18
OverTime
source


In [17]:
tmp = woe_iv(train, 'DailyRate', 'Attrition')
tmp.head()

0.044749109842095444


,binned,target,var_woe
0,3,0,-0.132883
1,5,0,-0.064854
2,4,0,0.08349
3,NaN,0,0
4,6,1,-0.078577


In [18]:
train['DailyRate'] = tmp['var_woe']
train.head()

,id,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EnvironmentSatisfaction,...,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Attrition,source
0,0,-0.338715,Travel_Frequently,-0.132883,Research & Development,24,3,Medical,1,4,...,1,10,2,3,10,0,7,8,0,synthetic
1,1,-0.101807,Travel_Rarely,-0.064854,Sales,8,3,Other,1,1,...,1,4,3,3,4,2,0,3,0,synthetic
2,2,-0.020844,Travel_Rarely,0.08349,Sales,26,3,Marketing,1,3,...,2,4,3,3,3,2,1,2,0,synthetic
3,3,-0.338715,Travel_Rarely,0,Research & Development,2,3,Medical,1,3,...,0,15,1,1,6,0,0,2,0,synthetic
4,4,0,Travel_Rarely,-0.078577,Research & Development,5,4,Medical,1,2,...,0,31,0,3,31,14,4,10,1,synthetic


In [19]:
tmp = woe_iv(train, 'DistanceFromHome', 'Attrition')
tmp.head()

0.02178420634360023


,binned,target,var_woe
0,NaN,0,0
1,5,0,-0.070899
2,NaN,0,0
3,1,0,-0.177526
4,3,1,-0.01844


In [20]:
train['DistanceFromHome'] = tmp['var_woe']
train.head()

,id,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EnvironmentSatisfaction,...,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Attrition,source
0,0,-0.338715,Travel_Frequently,-0.132883,Research & Development,0,3,Medical,1,4,...,1,10,2,3,10,0,7,8,0,synthetic
1,1,-0.101807,Travel_Rarely,-0.064854,Sales,-0.070899,3,Other,1,1,...,1,4,3,3,4,2,0,3,0,synthetic
2,2,-0.020844,Travel_Rarely,0.08349,Sales,0,3,Marketing,1,3,...,2,4,3,3,3,2,1,2,0,synthetic
3,3,-0.338715,Travel_Rarely,0,Research & Development,-0.177526,3,Medical,1,3,...,0,15,1,1,6,0,0,2,0,synthetic
4,4,0,Travel_Rarely,-0.078577,Research & Development,-0.01844,4,Medical,1,2,...,0,31,0,3,31,14,4,10,1,synthetic


In [21]:
train.YearsWithCurrManager.value_counts()

2     731
0     561
7     481
3     316
8     233
4     205
1     147
9     121
5      68
6      63
10     56
11     52
12     40
13     31
17     15
14     12
15     11
16      4
Name: YearsWithCurrManager, dtype: int64

In [22]:
tmp = woe_iv(train, 'HourlyRate', 'Attrition')
tmp.head()

0.016774516284387024


,binned,target,var_woe
0,1,0,-0.084
1,1,0,-0.084
2,6,0,-0.158674
3,NaN,0,0
4,NaN,1,0


In [23]:
train['HourlyRate'] = tmp['var_woe']
train.head()

,id,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EnvironmentSatisfaction,...,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Attrition,source
0,0,-0.338715,Travel_Frequently,-0.132883,Research & Development,0,3,Medical,1,4,...,1,10,2,3,10,0,7,8,0,synthetic
1,1,-0.101807,Travel_Rarely,-0.064854,Sales,-0.070899,3,Other,1,1,...,1,4,3,3,4,2,0,3,0,synthetic
2,2,-0.020844,Travel_Rarely,0.08349,Sales,0,3,Marketing,1,3,...,2,4,3,3,3,2,1,2,0,synthetic
3,3,-0.338715,Travel_Rarely,0,Research & Development,-0.177526,3,Medical,1,3,...,0,15,1,1,6,0,0,2,0,synthetic
4,4,0,Travel_Rarely,-0.078577,Research & Development,-0.01844,4,Medical,1,2,...,0,31,0,3,31,14,4,10,1,synthetic


In [24]:
tmp = woe_iv(train, 'MonthlyIncome', 'Attrition')
tmp.head()

0.2648794252235843


,binned,target,var_woe
0,1,0,0.912957
1,2,0,0.382559
2,4,0,-0.21637
3,5,0,-0.737667
4,NaN,1,0


In [25]:
train['MonthlyIncome'] = tmp['var_woe']
train.head()

,id,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EnvironmentSatisfaction,...,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Attrition,source
0,0,-0.338715,Travel_Frequently,-0.132883,Research & Development,0,3,Medical,1,4,...,1,10,2,3,10,0,7,8,0,synthetic
1,1,-0.101807,Travel_Rarely,-0.064854,Sales,-0.070899,3,Other,1,1,...,1,4,3,3,4,2,0,3,0,synthetic
2,2,-0.020844,Travel_Rarely,0.08349,Sales,0,3,Marketing,1,3,...,2,4,3,3,3,2,1,2,0,synthetic
3,3,-0.338715,Travel_Rarely,0,Research & Development,-0.177526,3,Medical,1,3,...,0,15,1,1,6,0,0,2,0,synthetic
4,4,0,Travel_Rarely,-0.078577,Research & Development,-0.01844,4,Medical,1,2,...,0,31,0,3,31,14,4,10,1,synthetic


In [26]:
tmp = woe_iv(train, 'MonthlyRate', 'Attrition')
tmp.head()

0.029820414195859252


,binned,target,var_woe
0,1,0,-0.035298
1,3,0,0.338292
2,5,0,-0.035298
3,4,0,-0.308895
4,7,1,-0.059395


In [27]:
train['MonthlyRate'] = tmp['var_woe']
train.head()

,id,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EnvironmentSatisfaction,...,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Attrition,source
0,0,-0.338715,Travel_Frequently,-0.132883,Research & Development,0,3,Medical,1,4,...,1,10,2,3,10,0,7,8,0,synthetic
1,1,-0.101807,Travel_Rarely,-0.064854,Sales,-0.070899,3,Other,1,1,...,1,4,3,3,4,2,0,3,0,synthetic
2,2,-0.020844,Travel_Rarely,0.08349,Sales,0,3,Marketing,1,3,...,2,4,3,3,3,2,1,2,0,synthetic
3,3,-0.338715,Travel_Rarely,0,Research & Development,-0.177526,3,Medical,1,3,...,0,15,1,1,6,0,0,2,0,synthetic
4,4,0,Travel_Rarely,-0.078577,Research & Development,-0.01844,4,Medical,1,2,...,0,31,0,3,31,14,4,10,1,synthetic


In [28]:
tmp = woe_iv(train, 'TotalWorkingYears', 'Attrition')
tmp.head()

0.07559932645211226


,binned,target,var_woe
0,5,0,-0.066208
1,1,0,0.312074
2,1,0,0.312074
3,7,0,-0.151147
4,NaN,1,0


In [29]:
train['TotalWorkingYears'] = tmp['var_woe']
train.head()

,id,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EnvironmentSatisfaction,...,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Attrition,source
0,0,-0.338715,Travel_Frequently,-0.132883,Research & Development,0,3,Medical,1,4,...,1,-0.066208,2,3,10,0,7,8,0,synthetic
1,1,-0.101807,Travel_Rarely,-0.064854,Sales,-0.070899,3,Other,1,1,...,1,0.312074,3,3,4,2,0,3,0,synthetic
2,2,-0.020844,Travel_Rarely,0.08349,Sales,0,3,Marketing,1,3,...,2,0.312074,3,3,3,2,1,2,0,synthetic
3,3,-0.338715,Travel_Rarely,0,Research & Development,-0.177526,3,Medical,1,3,...,0,-0.151147,1,1,6,0,0,2,0,synthetic
4,4,0,Travel_Rarely,-0.078577,Research & Development,-0.01844,4,Medical,1,2,...,0,0,0,3,31,14,4,10,1,synthetic


In [30]:
temp_s = pd.get_dummies(train, columns = ['Education',
'EnvironmentSatisfaction',
'JobInvolvement',
'JobLevel',
'JobSatisfaction',
'NumCompaniesWorked',
'PercentSalaryHike',
'PerformanceRating',
'RelationshipSatisfaction',
'WorkLifeBalance',
'TrainingTimesLastYear',
'YearsInCurrentRole',
'YearsSinceLastPromotion',
'YearsWithCurrManager',
'StockOptionLevel',
'BusinessTravel',
'Department',
'EducationField',
'Gender',
'JobRole',
'MaritalStatus',
'Over18',
'OverTime',
'source'])
temp_s.columns

Index(['id', 'Age', 'DailyRate', 'DistanceFromHome', 'EmployeeCount',
       'HourlyRate', 'MonthlyIncome', 'MonthlyRate', 'StandardHours',
       'TotalWorkingYears',
       ...
       'JobRole_Sales Executive', 'JobRole_Sales Representative',
       'MaritalStatus_Divorced', 'MaritalStatus_Married',
       'MaritalStatus_Single', 'Over18_Y', 'OverTime_No', 'OverTime_Yes',
       'source_original', 'source_synthetic'],
      dtype='object', length=166)

In [31]:
temp_s.head()

,id,Age,DailyRate,DistanceFromHome,EmployeeCount,HourlyRate,MonthlyIncome,MonthlyRate,StandardHours,TotalWorkingYears,...,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,Over18_Y,OverTime_No,OverTime_Yes,source_original,source_synthetic
0,0,-0.338715,-0.132883,0,1,-0.084,0.912957,-0.035298,80,-0.066208,...,0,0,0,1,0,1,0,1,0,1
1,1,-0.101807,-0.064854,-0.070899,1,-0.084,0.382559,0.338292,80,0.312074,...,0,1,0,1,0,1,1,0,0,1
2,2,-0.020844,0.08349,0,1,-0.158674,-0.21637,-0.035298,80,0.312074,...,1,0,1,0,0,1,1,0,0,1
3,3,-0.338715,0,-0.177526,1,0,-0.737667,-0.308895,80,-0.151147,...,0,0,0,1,0,1,1,0,0,1
4,4,0,-0.078577,-0.01844,1,0,0,-0.059395,80,0,...,0,0,0,0,1,1,0,1,0,1


In [32]:
features = list(temp_s.columns)
features.remove('id')
features.remove('Attrition')
features.remove('EmployeeCount')
features.remove('StandardHours')

target = 'Attrition'

In [33]:
features

['Age',
 'DailyRate',
 'DistanceFromHome',
 'HourlyRate',
 'MonthlyIncome',
 'MonthlyRate',
 'TotalWorkingYears',
 'YearsAtCompany',
 'Education_1',
 'Education_2',
 'Education_3',
 'Education_4',
 'Education_5',
 'Education_15',
 'EnvironmentSatisfaction_1',
 'EnvironmentSatisfaction_2',
 'EnvironmentSatisfaction_3',
 'EnvironmentSatisfaction_4',
 'JobInvolvement_1',
 'JobInvolvement_2',
 'JobInvolvement_3',
 'JobInvolvement_4',
 'JobLevel_1',
 'JobLevel_2',
 'JobLevel_3',
 'JobLevel_4',
 'JobLevel_5',
 'JobLevel_7',
 'JobSatisfaction_1',
 'JobSatisfaction_2',
 'JobSatisfaction_3',
 'JobSatisfaction_4',
 'NumCompaniesWorked_0',
 'NumCompaniesWorked_1',
 'NumCompaniesWorked_2',
 'NumCompaniesWorked_3',
 'NumCompaniesWorked_4',
 'NumCompaniesWorked_5',
 'NumCompaniesWorked_6',
 'NumCompaniesWorked_7',
 'NumCompaniesWorked_8',
 'NumCompaniesWorked_9',
 'PercentSalaryHike_11',
 'PercentSalaryHike_12',
 'PercentSalaryHike_13',
 'PercentSalaryHike_14',
 'PercentSalaryHike_15',
 'PercentSala

In [34]:
temp_s['Age'] = temp_s['Age'].astype(np.float16)
temp_s['DailyRate'] = temp_s['DailyRate'].astype(np.float16)
temp_s['DistanceFromHome'] = temp_s['DistanceFromHome'].astype(np.float16)
temp_s['HourlyRate'] = temp_s['HourlyRate'].astype(np.float16)
temp_s['MonthlyIncome'] = temp_s['MonthlyIncome'].astype(np.float16)
temp_s['MonthlyRate'] = temp_s['MonthlyRate'].astype(np.float16)
temp_s['TotalWorkingYears'] = temp_s['TotalWorkingYears'].astype(np.float16)
#Age, DailyRate, DistanceFromHome, HourlyRate, MonthlyIncome, MonthlyRate, TotalWorkingYears

In [35]:
clfs = []
scores = []


kf = StratifiedKFold(n_splits=10, random_state=0, shuffle=True)
for train_index, val_index in kf.split(temp_s, y=train['Attrition']):
    X_train, X_val = temp_s[features].loc[train_index], temp_s[features].loc[val_index]
    y_train, y_val = train[target][train_index], train[target][val_index]
    
    clf = LGBMClassifier(n_estimators=150, categorical_feature=[1, 3, 6, 9, 13, 15, 19, 20, 33], metric='auc')
    clf.fit(X_train.values, y_train, eval_set=[(X_val, y_val)], verbose=False)
    preds = clf.predict_proba(X_val.values)
    
    clfs.append(clf)
    scores.append(roc_auc_score(y_val, preds[:, 1]))
print(f'mean score across all folds: {np.mean(scores)}')

mean score across all folds: 0.8397160699936809


In [36]:
for i in clf.feature_importances_.argsort()[::-1]:
    print(features[i], clf.feature_importances_[i]/clf.feature_importances_.sum())

YearsAtCompany 0.060444444444444446
Age 0.048
MonthlyIncome 0.04644444444444444
MonthlyRate 0.04622222222222222
DistanceFromHome 0.04377777777777778
OverTime_No 0.024666666666666667
JobSatisfaction_4 0.018444444444444444
JobSatisfaction_1 0.016888888888888887
EnvironmentSatisfaction_1 0.016666666666666666
StockOptionLevel_0 0.01622222222222222
RelationshipSatisfaction_1 0.016
Education_3 0.014888888888888889
BusinessTravel_Travel_Frequently 0.014222222222222223
EnvironmentSatisfaction_4 0.013777777777777778
RelationshipSatisfaction_4 0.013777777777777778
WorkLifeBalance_3 0.013555555555555555
JobInvolvement_3 0.013555555555555555
YearsWithCurrManager_0 0.012888888888888889
TrainingTimesLastYear_3 0.012444444444444444
JobInvolvement_1 0.012444444444444444
JobRole_Research Scientist 0.012444444444444444
source_original 0.012
TrainingTimesLastYear_2 0.011555555555555555
JobSatisfaction_3 0.011333333333333334
JobInvolvement_4 0.009777777777777778
Gender_Female 0.009777777777777778
NumCompa

In [37]:
from catboost import CatBoostClassifier

scores = []
kf = StratifiedKFold(n_splits=10, random_state=1, shuffle=True)
for train_index, val_index in kf.split(temp_s, y=train['Attrition']):
    X_train, X_val = temp_s[features].loc[train_index], temp_s[features].loc[val_index]
    y_train, y_val = train[target][train_index], train[target][val_index]

    clf = CatBoostClassifier(iterations=200)
    clf.fit(X_train, y_train, eval_set=(X_val, y_val), verbose=False)
    
    preds = clf.predict_proba(X_val.values)[:, 1]
    clfs.append(clf)
    scores.append(roc_auc_score(y_val, preds))
print(f'mean auc across all folds: {np.mean(scores)}')

mean auc across all folds: 0.8540857602022109
